<a href="https://colab.research.google.com/github/bosonphoton/vLLM/blob/main/HaluEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install vllm datasets tqdm rouge_score transformers

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━

In [4]:
!pip install datasets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.


In [1]:
from vllm import LLM, SamplingParams
from datasets import load_dataset
from tqdm import tqdm
import re
from rouge_score import rouge_scorer
from transformers import AutoTokenizer

INFO 05-13 00:23:43 [__init__.py:239] Automatically detected platform cuda.


In [2]:
# Load the HaluEval dataset
dataset = load_dataset("pminervini/HaluEval", 'qa')
knowledge = dataset["data"]["knowledge"]
question = dataset["data"]["question"]
correct_answer = dataset["data"]["right_answer"]
hallucinated_answer = dataset["data"]["hallucinated_answer"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.88k [00:00<?, ?B/s]

data-00000-of-00001.parquet:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

Generating data split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [3]:
# ----------------------------
# 1. Format Qwen prompts
# ----------------------------
def format_prompt(q, tokenizer):
    prompt = "Give the final answer wrapped in <<>>. For instance <<salt>>"
    chat = [
      {"role": "system", "content": "You are a reasoning agent."},
      {"role": "user", "content": q + prompt},
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# ----------------------------
# 2. Extract text inside << >>
# ----------------------------
def extract_answer(text):
    match = re.search(r"<<(.*?)>>", text)
    return match.group(1).strip() if match else text.strip()

def rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    score = scorer.score(reference, candidate)
    return score['rouge1'].fmeasure

In [4]:
# ----------------------------
# 3. Load model with vLLM
# ----------------------------
llm = LLM(model="Qwen/Qwen3-0.6B", max_model_len=512, dtype="half")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

sampling_params = SamplingParams(
    temperature=0.7,
    max_tokens=1024
)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

WARNING 05-13 00:24:33 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-13 00:24:57 [config.py:717] This model supports multiple tasks: {'reward', 'embed', 'score', 'classify', 'generate'}. Defaulting to 'generate'.
WARNING 05-13 00:24:57 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 05-13 00:24:57 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='Qwen/Qwen3-0.6B', speculative_config=None, tokenizer='Qwen/Qwen3-0.6B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=512, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_b

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

INFO 05-13 00:25:20 [weight_utils.py:281] Time spent downloading weights for Qwen/Qwen3-0.6B: 17.504832 seconds
INFO 05-13 00:25:20 [weight_utils.py:315] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-13 00:25:22 [loader.py:458] Loading weights took 2.08 seconds
INFO 05-13 00:25:22 [model_runner.py:1140] Model loading took 1.1201 GiB and 20.217158 seconds
INFO 05-13 00:25:24 [worker.py:287] Memory profiling takes 1.39 seconds
INFO 05-13 00:25:24 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 05-13 00:25:24 [worker.py:287] model weights take 1.12GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 10.71GiB.
INFO 05-13 00:25:25 [executor_base.py:112] # cuda blocks: 6267, # CPU blocks: 2340
INFO 05-13 00:25:25 [executor_base.py:117] Maximum concurrency for 512 tokens per request: 195.84x
INFO 05-13 00:25:30 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 05-13 00:26:04 [model_runner.py:1592] Graph capturing finished in 34 secs, took 0.31 GiB
INFO 05-13 00:26:05 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 42.11 seconds


In [6]:
X = question[:10]
y = correct_answer[:10]
k = knowledge[:10]

# Format prompts
prompts = [format_prompt(q, tokenizer) for q in X]
for p in prompts:
  print(p)
  print(k)

# ----------------------------
# 5. Generate predictions
# ----------------------------
outputs = llm.generate(prompts=prompts, sampling_params=sampling_params)

# ----------------------------
# 6. Compare predictions to ground truth
# ----------------------------

print("\n--- Results ---\n")
final_accuracy = 0
for i, output in enumerate(outputs):
    raw = output.outputs[0].text
    y_pred = extract_answer(raw)
    gt = y[i]
    rouge_f1 = rouge(gt,y_pred)

    print(f"Question: {X[i]}\n")
    print(f"Reasoning: {raw}\n")
    print(f"Ground Truth: {gt}\n")
    print(f"AI Output: {y_pred}\n")
    print(f"Rouge: {rouge_f1}\n")

    print("---------------------------")

    final_accuracy += rouge_f1


print("FINAL ACCURACY ", final_accuracy / len(X))



<|im_start|>system
You are a reasoning agent.<|im_end|>
<|im_start|>user
Which magazine was started first Arthur's Magazine or First for Women?Give the final answer wrapped in <<>>. For instance <<salt>><|im_end|>
<|im_start|>assistant

["Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.First for Women is a woman's magazine published by Bauer Media Group in the USA.", 'The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group.The Oberoi Group is a hotel company with its head office in Delhi.', 'Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician. Goertz is known for her satirical songs based on various pop culture topics. Her videos are posted on YouTube under the name of Cossbysweater.Milhouse Mussolini van Houten is a fictional character featured in the animated television series "The Simpsons", voiced by Pamela Hayden, and created by Matt Groe

Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


--- Results ---

Question: Which magazine was started first Arthur's Magazine or First for Women?

Reasoning: <think>
Okay, the user is asking which magazine was started first between Arthur's Magazine and First for Women. Let me start by recalling what I know about these magazines.

First, Arthur's Magazine was founded in 1908 by notable figures. I remember that it was around the time when the magazine was established, and it's associated with the Arthurian legends. On the other hand, First for Women was founded in 1931 by Doris, a prominent figure in the feminist movement. 

Now, I need to compare the years. Arthur's Magazine was created in 1908, while First for Women was founded in 1931. Since 1908 is earlier than 1931, Arthur's Magazine must have been started first. 

I should double-check to ensure there's no confusion with other magazines or events. Both magazines are indeed well-known and have their own histories, so the answer should be straightforward. The user provided an ex